In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ds=pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
ds.head()

In [ ]:
ds.info()

In [ ]:
ds.isnull().sum()

In [ ]:
x=ds.iloc[:,1:-1].values
y=ds.iloc[:,-1].values

In [ ]:
x

In [ ]:
#dealin with missin data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(x[:,8:9])
x[:,8:9]=imputer.transform(x[:,8:9])

In [ ]:
#checkin' the 'bmi' column
np.array(x[:,8])

In [ ]:
#encodin' the categorical data
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,0]=le.fit_transform(x[:,0])
x[:,4]=le.fit_transform(x[:,4])

In [ ]:
x

In [ ]:
#using onehotencoder for rest of the features
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5,6,9])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
#recheckin' the data
x

In [ ]:
#splittin' the dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [ ]:
#feature scaling the data
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clsfr=LogisticRegression()
clsfr.fit(x_train,y_train)

In [ ]:
y_pred=clsfr.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score #confusion matrix and accuracy score
cm=confusion_matrix(y_test,y_pred)
ac=accuracy_score(y_test,y_pred)
print(cm,ac)

In [ ]:
#checing cross validation score
from sklearn.model_selection import cross_val_score
acrcy=cross_val_score(estimator=clsfr,X=x_train,y=y_train,cv=10)
print('Accuracy',acrcy.mean()*100)
print('Standard Deviation {:.2f}%'.format(acrcy.std()*100))